In [ ]:
import code_share
from code_share import TRAINING_SET_PATH, TRAINING_LABEL_PATH, OUT_FILE_NAME, TESTING_SET_PATH, TESTING_LABEL_PATH, FILENAME_LIST_PATH, INPUT_SAVE_MODEL_PATH, NUM_EPCHS 
from code_share import MCC, save_train_history, evaluate_model

import os
import warnings
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime as DT
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, AveragePooling2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
import tensorflow.keras.backend as K
from sklearn.metrics import f1_score, precision_score, recall_score, matthews_corrcoef
import pandas as pd

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

warnings.filterwarnings('ignore')
matplotlib.use("Agg")
 
# Function to build the CNN model
def build_cnn_model(input_shape):
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(11, 11), strides=(4, 4), padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid'))

    # Flatten the output
    model.add(Flatten())

    # 1st Fully Connected Layer
    model.add(Dense(4096, activation='relu'))

    # 2nd Fully Connected Layer
    model.add(Dense(4096, activation='relu'))

    # Output Layer
    model.add(Dense(12, activation='softmax'))  # Assuming binary classification; change output layer size for different number of classes

    return model


# Load and preprocess training data
x_sample = np.load(TRAINING_SET_PATH)
x_sample = x_sample.reshape((x_sample.shape[0], 100, 100, 1))

y_label = np.load(TRAINING_LABEL_PATH)
y_label = to_categorical(y_label) 

# Build CNN model
model = build_cnn_model(x_sample[0].shape)
model.summary()

# Compile model
adam = Adam(learning_rate=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy', MCC])

# Define callbacks
best_model_path = OUT_FILE_NAME + "_best_weights.keras"
estop = EarlyStopping(monitor="val_MCC", patience=60, verbose=0, mode="max")
checkpoint = ModelCheckpoint(best_model_path, monitor="val_MCC", verbose=0, save_best_only=True, mode="max")
tb_callback = TensorBoard(log_dir=OUT_FILE_NAME + '_logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_metadata=None, update_freq='epoch')

# Train model
train_history = model.fit(x_sample, y_label, validation_split=0.25, epochs=NUM_EPCHS, batch_size=24, verbose=1, callbacks=[estop, checkpoint, tb_callback])

# Save training log
with open(OUT_FILE_NAME + "_tranning_message", 'w') as out:
    for step, (i, mcc) in enumerate(zip(train_history.history['accuracy'], train_history.history['MCC']), 1):
        out.write(f'Epoch:\t{step}/{NUM_EPCHS}\tLoss:\t{train_history.history["loss"][step-1]:.6f}\tAcc:\t{i:.6f}\tMCC:\t{mcc:.6f}\tVal_Loss:\t{train_history.history["val_loss"][step-1]:.6f}\tVal_Acc:\t{train_history.history["val_accuracy"][step-1]:.6f}\tVal_MCC:\t{train_history.history["val_MCC"][step-1]:.6f}\n')

# Save final model
final_model = OUT_FILE_NAME + ".weights.h5"
model.save_weights(final_model)

# Plot training history
save_train_history(train_history, "accuracy", "val_accuracy", OUT_FILE_NAME + "-acc")
save_train_history(train_history, "loss", "val_loss", OUT_FILE_NAME + "-loss")

# Load testing data and labels
test_sample = np.load(TESTING_SET_PATH)
test_sample = test_sample.reshape((test_sample.shape[0], 100, 100, 1))

test_label = np.load(TESTING_LABEL_PATH)
test_label_compare = test_label
test_label = to_categorical(test_label)

test_sample_title = np.load(FILENAME_LIST_PATH)
 
model = build_cnn_model(test_sample[0].shape)

evaluate_model(model, "AlexNet", test_sample, test_sample_title, test_label, test_label_compare)